In [1]:
# Import Pandas
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
import pickle
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import *
from tensorflow.keras.applications import *
from tensorflow.keras.models import Model 
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# Load Movies Metadata
df = pd.read_excel('DFI -Ticketing.xlsx', sheet_name=None)

df = pd.DataFrame(df['Export - Tickets'])


In [4]:
df = df.dropna(subset=['Incident-Subject','Ticket-Description'])


In [5]:
df['Incident-Subject'][0]

" Bonsoir  \xa0  Notre outil de supervision a remonté l'alerte suivante:  \xa0  Client:  Serveur: FRDAPLANIF1  Adresse IP: 170.237.25.77  Service: Service: _SMAOpConMinosService_\xa0  Note: Serveur Windows  \xa0  Retour du check: SMAOpConMinosService: Stopped  \xa0  \xa0  Cordialement, Adil  "

In [6]:
def add_questions(df,question):

  tickets_questions = df['Incident-Subject'].copy()
  tickets_questions = tickets_questions.to_frame()

  x=str(question)
  
  x = pd.DataFrame([[x]], columns=['Incident-Subject'])
  tickets_questions=tickets_questions.append(x, ignore_index=True)

  return tickets_questions



In [7]:
def answer_ticket(df,prediction,question):

    df_team = df.loc[df["Team"]==prediction]
    tickets_questions = df_team['Incident-Subject'].copy()
    tickets_questions = tickets_questions.to_frame()

    x=str(question)
    
    x = pd.DataFrame([[x]], columns=['Incident-Subject'])
    tickets_questions=tickets_questions.append(x, ignore_index=True)

    #Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
    tfidf = TfidfVectorizer()
    
    #Construct the required TF-IDF matrix by fitting and transforming the data
    tfidf_matrix = tfidf.fit_transform(tickets_questions['Incident-Subject'])
    cosine_similarity = linear_kernel(tfidf_matrix, tfidf_matrix)
    sim_scores = list(enumerate(cosine_similarity[-1]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    max_scores = [i[0] for i in sim_scores if i[0]<len(sim_scores)-1]
    max_scores = max_scores[0:5]
    proposals = df['Ticket-Description'][max_scores]
    return proposals


In [8]:
question =  'Bonsoir     Notre outil de supervision a remonté l''alerte suivante:     Client:  Serveur: FRDAPLANIF1  Adresse IP: 170.237.25.77  Service: Service: _SMAOpConMinosService_   Note: Serveur Windows     Retour du check: SMAOpConMinosService: Stopped        Cordialement, Adil'


In [9]:
new_model = tf.keras.models.load_model("my_model.h5")

with open('tokenizer.pickle', 'rb') as handle:
  tokenizer = pickle.load(handle)

with open('lables.pickle', 'rb') as handle:
    lables_columns = pickle.load(handle)

question=np.array(question)
question = question.reshape(-1)
question = tokenizer.texts_to_sequences(question)
question = pad_sequences(question)
team_prediction =    lables_columns[np.argmax(new_model.predict(question))]
team_prediction

'PILOTAGE'

In [10]:
proposals = answer_ticket(df,team_prediction,question)
proposals

372      Bonjour     Je clôture ce ticket :     Travai...
1080                      le job n'est plus en message   
1022     Bonjour,     Backup ok pour  :     NANSP003  ...
27       Travail 377567/SABEXPLOIT/SAB06501 démarré le...
2151     Désactivation de la GPO qui bride le verouill...
Name: Ticket-Description, dtype: object